In [1]:
import os
import math
import numpy as np
import pandas as pd
from ast import literal_eval
import itertools
import arviz as az
import scipy.stats as st
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib.patches as  mpatches
pd.options.mode.chained_assignment = None  # default='warn'

from helper_functions import *

# Fixations: insert N_visible_obstacles & N_visible_drift_tiles

In [4]:
fix_data = pd.read_csv(f'eye_data/experimental_eye_data_fixations.csv', index_col=False)
fix_data = fix_data.drop(labels=["Unnamed: 0"], axis = 1)
fix_data

,index,time_tag,TimeTag,LeftEyeX,LeftEyeY,LeftPupilDiameter,RightEyeX,RightEyeY,RightPupilDiameter,DigitalIn,...,distance_to_spaceship_in_pixel,distance_to_spaceship,exploring_fixation,N_visible_drift_tiles,N_visible_obstacles,ID,level,drift,input_noise,level_difficulty
0,362,0.1810,8555.6925,153.147766,280.140991,51.062500,138.066833,263.747314,59.636719,16777215.0,...,135.087674,2.853091,0,0.0,0.0,AE07EM1,2,T,N,easy
1,3010,1.5050,8557.0165,-61.182617,3.833374,57.843750,-26.071381,-5.930054,63.949219,16777215.0,...,259.090381,5.460004,1,0.0,0.0,AE07EM1,2,T,N,easy
2,3485,1.7425,8557.2540,-6.799438,220.455322,55.980469,-5.339569,193.183472,62.375000,16777215.0,...,48.659982,1.028453,0,0.0,0.0,AE07EM1,2,T,N,easy
3,3902,1.9510,8557.4625,-17.254700,282.880615,51.070312,-11.294189,258.508545,57.000000,16777215.0,...,32.246453,0.681586,0,0.0,0.0,AE07EM1,2,T,N,easy
4,4835,2.4175,8557.9290,-6.512329,-155.344238,55.539062,21.574677,-168.864441,60.718750,16777215.0,...,414.128452,8.686578,1,0.0,0.0,AE07EM1,2,T,N,easy
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
96258,51718,25.8590,22780.7710,-270.271942,218.176819,60.839844,31.836182,207.793091,63.933594,16777215.0,...,136.895237,2.891203,0,0.0,0.0,ZC12VE,3,T,S,medium
96259,53691,26.8455,22781.7575,-372.090424,332.374817,61.265625,-24.934143,294.829834,65.484375,16777215.0,...,219.340530,4.626289,1,0.0,0.0,ZC12VE,3,T,S,medium
96260,54138,27.0690,22781.9810,-206.885529,390.400208,60.238281,225.029602,274.641113,65.113281,16777215.0,...,80.573877,1.702650,0,0.0,0.0,ZC12VE,3,T,S,medium
96261,54625,27.3125,22782.2245,-207.103180,351.743469,58.187500,196.808167,253.705322,62.445312,16777215.0,...,53.544386,1.131662,0,0.0,0.0,ZC12VE,3,T,S,medium


In [42]:
N_visible_obs_col = []
N_visible_drift_col = []

# create groups that reflect files in data
groups = fix_data.groupby(["ID", "level", "drift", "input_noise"])

for key, grp in groups:
    #print(f'\nGroup: {key}')
    ID = key[0]
    level = key[1]
    drift = key[2]
    input_noise = key[3]
    
    input_file_name = f'{ID}_output_{level}{drift}{input_noise}_done_'
    
    path = f'experimental_data/{ID}/data/'
    
    file_found = False
    
    for subdir, dirs, files in os.walk(path):
        for data_file in files:
            if input_file_name in str(data_file):
                
                file_found = True
    
    if file_found:        
        input_data = pre_process_input_data(pd.read_csv(f'{path}/{data_file}', index_col=False))
                
    for index, row in grp.iterrows():
        if file_found:
            closest_frame = input_data.iloc[(input_data['time_played'] - row.time_tag).abs().argsort()[0]]

            N_visible_obstacles = len(closest_frame.visible_obstacles)
            N_visible_drift = len(closest_frame.visible_drift_tiles)

            N_visible_obs_col.append(N_visible_obstacles)
            N_visible_drift_col.append(N_visible_drift)
            
        elif not file_found:
            N_visible_obs_col.append(np.nan)
            N_visible_drift_col.append(np.nan)

In [41]:
N_obs_recent = N_visible_obs_col.copy()

In [27]:
# should be: 96263; is: 95983; or: 95976
len(N_visible_obs_col)

96263

In [47]:
N_obs_diffs = fix_data.N_visible_obstacles - N_visible_obs_col
min(N_obs_diffs)

-18.0

In [39]:
N_obs_diffs[:20]

0     0.0
1     0.0
2     0.0
3     0.0
4     0.0
5     0.0
6     0.0
7     1.0
8     1.0
9     1.0
10    0.0
11    0.0
12    0.0
13    0.0
14   -1.0
15    0.0
16    0.0
17   -1.0
18   -1.0
19    0.0
Name: N_visible_obstacles, dtype: float64

In [34]:
def nextNonZeroIndex(array, start):
    nextItems = array[start+1:]
    for i, x in enumerate(nextItems):
        if x != 0:
            return 1 + start + i

In [ ]:
N_visible_obs_col = []
N_visible_drift_col = []

for index, row in fix_data.iterrows():
    
    input_file_name = f'{row.ID}_output_{row.level}{str(row.drift)[0]}{row.input_noise}_done_'
    
    path = f'experimental_data/{row.ID}/data/'
    
    file_found = False
    
    for subdir, dirs, files in os.walk(path):
        for data_file in files:
            if input_file_name in str(data_file):
                
                file_found = True
                
                #print(f'searching for {data_file}')
                
                input_data = pre_process_input_data(pd.read_csv(f'{path}/{data_file}', index_col=False))
                
                closest_frame = input_data.iloc[(input_data['time_played'] - row.time_tag).abs().argsort()[0]]
                
                N_visible_obstacles = len(closest_frame.visible_obstacles)
                N_visible_drift = len(closest_frame.visible_drift_tiles)
                
                N_visible_obs_col.append(N_visible_obstacles)
                N_visible_drift_col.append(N_visible_drift)
    
    if not file_found:
        N_visible_obs_col.append(np.nan)
        N_visible_drift_col.append(np.nan)

In [ ]:
fix_data.insert (37, 'N_visible_obstacles', N_visible_obs_col)
fix_data.insert (37, 'N_visible_drift_tiles', N_visible_drift_col)